# 18. 协程（1）
- 参考资料：
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/1190000009781688

# 两个前提概念

## 迭代器
- 可迭代(Iterable)：直接作用于for循环的变量
- 迭代器(Iterator)：不但可以作用于for循环，还可以被next调用
    - next：将迭代器的 next() 方法调用转为 next() 函数。也会将 next() 方法重命名为 \_\_next__()。
- 一个迭代器必定是一个可迭代对象，但一个可迭代对象不一定是迭代器
    - list是典型的可迭代对象，但不是迭代器，因为不可以被next调用
- 是否为迭代器，可通过isinstance判断
    - 修复 isinstance() 函数第二个实参中重复的类型。
    - 举例来说，isinstance(x, (int, int)) 会转换为 isinstance(x, int), isinstance(x, (int, float, int)) 会转换为 isinstance(x, (int, float))。
- Iterable可以通过iter()函数转换为Iterator
    - iter()：返回一个 iterator 对象。
    - 格式：iter(object[, sentinel])
    - 根据是否存在第二个实参，第一个实参的解释是非常不同的。
    - 如果没有第二个实参，object 必须是支持迭代协议（有 __iter__() 方法）的集合对象，或必须支持序列协议（有 __getitem__() 方法，且数字参数从 0 开始）。
        - 如果它不支持这些协议，会触发 TypeError。
    - 如果有第二个实参 sentinel，那么 object 必须是可调用的对象。这种情况下生成的迭代器，每次迭代调用它的 __next__() 方法时都会不带实参地调用 object；如果返回的结果是 sentinel 则触发 StopIteration，否则返回调用结果。

In [2]:
# 可迭代对象 示例
# 类似这样的称为可迭代，但 l 不是迭代器
l = [i for i in range(5)]

for idx in l:
    print(idx)

0
1
2
3
4


In [3]:
# range是一个迭代器 示例
for i in range(5):
    print(i)

0
1
2
3
4


In [7]:
# isinstance() 示例
# 判断某个变量是否是一个实例

# collections 模块将在Python3.8中被弃用，所以应该从collections.abc这个模块中导入
from collections.abc import Iterable, Iterator

ll = [1, 2, 3, 4, 5]

# 判断是否可迭代
print(isinstance(ll, Iterable))

# 判断是否为迭代器
print(isinstance(ll, Iterator))

True
False


In [10]:
# iter()函数 示例
s = 'I love bingbing'

# 判断是否可迭代
print(isinstance(s, Iterable))
# 判断是否为迭代器
print(isinstance(s, Iterator))

# 转换
s_iter = iter(s)

# 判断是否可迭代
print(isinstance(s_iter, Iterable))
# 判断是否为迭代器
print(isinstance(s_iter, Iterator))

True
False
True
True


## 生成器（$Generator$）
- 生成器：一边循环一边计算下一个元素的机制或算法
- 生成器是一个用于创建迭代器的简单而强大的工具。
- 生成器需要满足三个条件：
    - 每次调用都生产出for循环需要的下一个元素
    - 如果到达最后一个，报出StopIteration异常
    - 可以被next()调用
- 生成器的写法类似标准的函数，但当它们要返回数据时会使用 yield 语句。
- 每次对生成器调用 next() 时，它会从上次离开位置恢复执行（它会记住上次执行语句时的所有数据值）。
- 如何制作生成器：
    1. 直接使用 [生成器表达式](https://docs.python.org/zh-cn/3.9/reference/expressions.html?highlight=yield#generator-expressions)
    2. 如果函数中包含[yield表达式](https://docs.python.org/zh-cn/3.9/reference/expressions.html?highlight=yield#yield-expressions)，则这个函数就叫生成器，并用next()函数调用，遇到yield表达式返回
- yield的执行方式类似于将函数进行了分段，然后每次调用生成器将一点一点地执行函数

In [14]:
# 直接使用生成器 示例

l = [x*x for x in range(5)] # 放在中括号里是列表生成器
g = (x*x for x in range(5)) # 放在小括号里是生成器表达式

print(type(l))
print(type(g))  

<class 'list'>
<class 'generator'>


In [15]:
# 普通函数 示例

def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None

odd()

Step 1
Step 2
Step 3


In [22]:
# yield 示例1

# 改造为生成器，由yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3

# 调用生成器时需要使用next()函数
one = next(odd())
print(one)

two = next(odd())
print(two)

three = next(odd())
print(three)

# 此处会出现问题，因为每次调用会重新打开生成器

Step 1
1
Step 1
1
Step 1
1


In [23]:
# yield 示例2
# 上面示例的修改
# 改造为生成器，由yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3

# 此处加入一个实例来表示打开同一个生成器
g = odd()

# 调用生成器时需要使用next()函数
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [24]:
# 使用for循环调用生成器 示例1

# 正常的for循环
def fib(max):
    # 斐波那契额数列的生成器写法
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        print(b)
        a, b = b, a+b # 注意写法
        n += 1
        
    return 'Done'

fib(5)

1
1
2
3
5


'Done'

In [25]:
# 使用for循环调用生成器 示例2

# 调用生成器的for循环
def fib(max):
    # 斐波那契额数列的生成器写法
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        yield b
        a, b = b, a+b # 注意写法
        n += 1
    
    #需要注意，报出异常是的返回值是return的返回值
    return 'Done'

g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

In [26]:
# 使用for循环调用生成器 示例3

ge = fib(10)
'''
生成器的典型用法是在for循环中使用
比较常用的典型生成器就是range()
'''
for i in ge: #在for循环中使用生成器
    print(i)

1
1
2
3
5
8
13
21
34
55


# 协程（$Coroutine$）
- 历史历程
    - 3.4引入协程，用yield实现
    - 3.5引入协程语法
    - 实现的协程比较好的包有：
        - asyncio
        - tornado
        - gevent（包含greenlet的所有功能）
        - greenlet（较老，是gevent的底层，主要控制协程的切换）
- 定义：协程 是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序。
- 从技术角度讲，协程就是一个你可以暂停执行的函数，或者干脆把协程理解成生成器
- 协程的实现：
    - yield返回
    - send调用
- 协程的四个状态
    - inspect.getgeneratorstate(…) 函数确定，该函数会返回下述字符串中的一个：
        - GEN_CREATED：等待开始执行
        - GEN_RUNNING：解释器正在执行
        - GEN_SUSPENED：在yield表达式处暂停
        - GEN_CLOSED：执行结束
    - next预激（prime)
- 协程终止
    - 协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象）
    - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的 None 和 Ellipsis 等常量经常用作哨符值==。
- yield from
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止会发出StopIteration异常，异常对象的vlaue属性保存返回值
    - 使用yield from从内部捕获StopIteration异常
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给自生成器
        - 子生成器在把产出的值发给调用放
        - 自生成器在最后，解释器会抛出StopIteration，并且把返回值附加到异常对象上
- 对于CPU和操作系统来说，协程是不存在的，协程只会被看做一条线程中的多个任务正在来回切换执行。
    - 因为只是一条线程在执行，所以不会对CPU产生负担，而且也不会有数据共享问题。

In [27]:
# 协程 示例1
def simple_coroutine():
    print('-> start')
    x = yield
    print('-> recived', x)

# 主线程
sc = simple_coroutine()
print(1111)

# 可以使用sc.send(None)，效果一样
next(sc) # 预激

print(2222)
sc.send('bingbing') # 发送给“x”

1111
-> start
2222
-> recived bingbing


StopIteration: 

In [28]:
# 协程的四种状态 示例

def simple_coroutine(a):
    print('-> start')

    b = yield a
    print('-> recived', a, b)

    c = yield a + b
    print('-> recived', a, b, c)

# runc
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6) # 5, 6
print(bb)
cc = sc.send(7) # 5, 6, 7
print(cc)

-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [29]:
# yield from 示例

def gen():
    for c in 'AB':
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from 'AB'

print(list(gen_new()))

['A', 'B']
['A', 'B']


In [30]:
# 委派生成器 示例
from collections import namedtuple

'''
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine）在 yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为 results[key] 赋值的语句还没有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''
ResClass = namedtuple('Res', 'count average')


# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None

    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()


# 客户端代码
def client():
    process_data = {
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)

# run
client()

{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


## 使用 greenlet 实现协程
- 此处主要实现的是一个线程中任务的切换，不会在任务阻塞时自动切换到其他任务，但这符合协程的定义，也算是一个协程

In [4]:
from greenlet import greenlet
import time

def mission1():
    print('我是任务1-1。')
    g2.switch()  # 切换到任务2
    print('我是任务1-2。')


def mission2():
    print('我是任务2-1。')
    time.sleep(3)  # 此处阻塞，但是不会自动切换为任务1，所以等了3秒
    print('我是任务2-2。')
    g1.switch()  # 切换到任务1
    


g1 = greenlet(mission1)
g2 = greenlet(mission2)

# 切换到任务1
g1.switch()

我是任务1-1。
我是任务2-1。
我是任务2-2。
我是任务1-2。


## 使用 gevent 实现协程
- gevent 模块会自动检测阻塞事件，遇到阻塞会自动切换任务
- 但是有些阻塞 gevent 模块并不认识。比如：不认识 time.sleep()，但可以使用 gevent.sleep()
- 可以手动让 gevent 认识一些阻塞

In [5]:
import gevent
import time

def mission1():
    print('我是任务1-1。')
    print('我是任务1-2。')


def mission2():
    print('我是任务2-1。')
    gevent.sleep(3)  # 此处阻塞，自动切换任务
    print('我是任务2-2。')


def mission3():
    print('我是任务3-1。')
    time.sleep(3)  # 此处阻塞，但是不会自动切换为任务1，所以等了3秒
    print('我是任务3-2。')


m1 = gevent.spawn(mission1)
m2 = gevent.spawn(mission2)
m3 = gevent.spawn(mission3)

# 阻塞
m1.join()  # 阻塞至 m1 结束
m2.join()  # 阻塞至 m2 结束
m3.join()  # 阻塞至 m3 结束

我是任务1-1。
我是任务1-2。
我是任务2-1。
我是任务3-1。
我是任务3-2。
我是任务2-2。


### 使用 gevent 中的 monkey 模块来让 gevevt 认识阻塞

In [8]:
# 让 gevevt 认识阻塞
# 要先导入 monkey 模块
from gevent import monkey
# 导入后直接使用下面的语句
monkey.patch_all()
# 然后再导入其他会引发阻塞的模块
import time
import gevent

def mission1():
    print('我是任务1-1。')
    time.sleep(5)  # 此处阻塞，自动切换任务
    print('我是任务1-2。')


def mission2():
    print('我是任务2-1。')
    gevent.sleep(4)  # 此处阻塞，自动切换任务
    print('我是任务2-2。')


def mission3():
    print('我是任务3-1。')
    time.sleep(3)  # 此处阻塞，自动切换任务
    print('我是任务3-2。')


m1 = gevent.spawn(mission1)
m2 = gevent.spawn(mission2)
m3 = gevent.spawn(mission3)

# 阻塞
m1.join()  # 阻塞至 m1 结束
m2.join()  # 阻塞至 m2 结束
m3.join()  # 阻塞至 m3 结束

我是任务1-1。
我是任务2-1。
我是任务3-1。
我是任务3-2。
我是任务2-2。
我是任务1-2。
